<img src='https://ibm.box.com/shared/static/mpiif2qy64af6ikycme4510d6tpgv5zy.png' width=250>
<img style="position:relative;width:30%" src='https://spark.apache.org/docs/0.9.0/img/spark-logo-hd.png'>
<img style="position:relative;width:30%", src='http://spark.apache.org/docs/latest/img/graphx_logo.png'>
<img style="position:relative;width:30%", src='https://pmcdeadline2.files.wordpress.com/2015/08/amazon-featured-image.jpg?w=446&h=299&crop=1'>
<h4 style="text-align: center;">Author: [Ehsan M. Kermani](http://ca.linkedin.com/in/ehsanmkermani)</h3>

># Amazon Graph Computation Tutorial

The complete dataset is a graph of Amazon product reviews from $1995$ to $2003$ with $2$M nodes representing customers and products, and $10$M edges representing reviews. The data was originally collected in 2003 by Jure Leskovec and published by the Stanford Network Analysis Project (SNAP) for research (https://snap.stanford.edu/data/). All real customer data has been removed, including the contents of each review. Customer names have been synthetically generated.

### What this tutorial is about?

In this tutorial, we would like to answer some of basic and important questions about [Amazon product co-purchasing network, March 02 2003](https://snap.stanford.edu/data/amazon0302.txt.gz) dataset, using **GraphX** which is the [Apache Spark](http://spark.apache.org/) graph processing library with its Scala API.

More precisely, we want to see how to define the notion of *cluster* in products and purchases? and finally how to compute the measure of *importance* of products with page rank algorithm?

Meanwhile, we will replicate some of the results [here](https://snap.stanford.edu/data/amazon0302.html) about the properties of the dataset represented as a graph.   

To do so, we quickly review some important concepts of graph theory.

### Dowload and unzip data

In [1]:
import sys.process._
import java.net.URL
import java.io.File
import java.io.BufferedReader
import java.io.InputStreamReader
import java.util.zip.GZIPInputStream
import java.io.FileInputStream

def fileDownloader(url: String, filename: String) = {
    new URL(url) #> new File(filename) !!
}

class BufferedReaderIterator(reader: BufferedReader) extends Iterator[String] {
  override def hasNext() = reader.ready
  override def next() = reader.readLine()
}

object GzFileIterator {
  def apply(file: java.io.File, encoding: String) = {
    new BufferedReaderIterator(
      new BufferedReader(
        new InputStreamReader(
          new GZIPInputStream(
            new FileInputStream(file)), encoding)))
  }
}

def printToFile(f: java.io.File)(op: java.io.PrintWriter => Unit) {
  val p = new java.io.PrintWriter(f)
  try { op(p) } finally { p.close() }
}

println("Downloading the zip file")
fileDownloader("https://snap.stanford.edu/data/amazon0302.txt.gz", "/resources/data/amazon0302.txt.gz")
println("Reading and unzipping")
val iterator = GzFileIterator(new java.io.File("/resources/data/amazon0302.txt.gz"), "UTF-8")
println("Writing to text file")
printToFile(new File("/resources/data/amazon0302.txt")) { p =>
  iterator.foreach(p.println)
}

Reading and unzipping
Writing to text file


### Directed graph 

A [graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29) is a mathematical entity with two sets of *vertices* (nodes) and *edges* that is connecting the vertices. If edges have directions, the graph is called [*directed* graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29#Directed_graph). It is possible that two vertices have multiple directed edges between them, as shown below

<img style="position:relative;width:50%" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Directed_graph.svg" >

Now, we would like to represent our data as a directed graph, where *vertices* are Amazon **products** and there is an *edge* between product $i$ and product $j,$ when product $i$ is *frequently* co-purchased with product $j.$ 

### GraphX

To contruct our `Graph` object, we should first import *GraphX* and all of its components. Then, in our case, the most convenient way for contructing our graph is with `GraphLoader` (singleton) factory object and calling `edgeListFile` which takes an input a text file with lines containing *FromNodeId* and *ToNodeId* representing a directed edge, with canonical orientation i.e. whether the orientation of edges is in the positive direction of *FromNodeId* $\longrightarrow$ *ToNodeId*.

For our purpose, we also need to provide `PartitionStragety` by invoking `partitionBy` method, because `GraphX` distributes a graph based on **vertex-cut** not *edge-cut* to optimize communications and storage.

<img style="position:relative;width:70%" src="https://spark.apache.org/docs/0.9.0/img/edge_cut_vs_vertex_cut.png" >

In [2]:
import org.apache.spark.graphx._

val amazonGraph = GraphLoader.edgeListFile(sc=sc,
                                           path="/resources/data/amazon0302.txt",
                                           canonicalOrientation=true).partitionBy(PartitionStrategy.RandomVertexCut)

#### How many products and co-purchase relations are in our data set?

We can obtain the *number* of vertices and edges of our `amazonGraph`, by accessing the fields `numVertices` and `numEdges` and verify the claimed numbers [here](https://snap.stanford.edu/data/amazon0302.html), as follows

In [3]:
println("Number of vertices: " + amazonGraph.numVertices)
println("Number of edges: " + amazonGraph.numEdges)

Number of vertices: 262111
Number of edges: 1234877


Similarly, we can access other features of our graph such as the `vertices`, `edges`, `inDegrees` and `outDegrees`.

We can also `cache` our `amazonGraph` (since the graph is small) to make the rest of our computations faster.

In [4]:
amazonGraph.cache()

org.apache.spark.graphx.impl.GraphImpl@5243a3ba

#### Find (the number of) "clusters" of products

To analyze more, we can find those products that are frequently co-purchased together and form a "cluster" in order to, for example, recommend "related" products when someone buys a product or even balance the supply-demand, etc. There are number of ways we can define what we mean by a cluster in a graph. The simplest cluster is a triangle.

A **triangle** in a (directed) graph contains three vertices (products here) where any two of them is connected by an (directed) edge 
i.e. a (directed) cycle of length 3. In other words, a vertex is part of a triangle when there are two 
adjacent vertices connected by an edge. In fact, a triangle can be considered as a small
cluster of related objects (products frequently purchased toghether), therefore, counting the 
number of triangles provides a measure for the number of these small clusters.

For example, (B, C, E) or (B, D, E) are triangles below. Nothing should stop up from generalizing and considering *squares* such as (B, C, E, D) below (without directions, of course), etc. but then counting the exact number will become more difficult.

<img style="position:relative;width:50%" src="http://www.cse.chalmers.se/edu/year/2015/course/DIT980/dag.svg" >


GraphX contains variety of operations such as `triangleCount()` that can be called on our 
`amazonGraph` object. The result is represented as a `Graph` where we can access its `vertices` or 
`edges`.

In [5]:
val triCounts = amazonGraph.triangleCount()

The result of calling `triCounts.vertices` is an RDD of pairs of 
**(VertexId, # triangles containing the Vertex)**. Therefore, we can count the total number 
of triangles, by suming up all the each vertex triangle count with `reduce`. Moreover, since each edge is counted *three* times (once for every vertex of a triangle), we need to divide by $3$ to find the exact total count. The result in fact, verifies the claim [here](https://snap.stanford.edu/data/amazon0302.html) about the number of triangles.

In [6]:
val totalTriCounts = triCounts.vertices.map(x => x._2).reduce(_ + _) / 3
println("Total number of triangles: " + totalTriCounts)

Total number of triangles: 717719


Following up in defining a cluster of products, we can consider a subset of products that any two are *related* i.e. there is a sequence of co-purchases from one to the other. Mathematically, that means there is a sequence of consecutive edges connecting them (path) and that is called a **connected component** in a graph.

In our directed graph `amazonGraph`, a *weakly connected component* (WCC) is a subset of vertices and edges, where any two vertices are related (by a path) *ignoring* the directions of edges. The example below shows a directed graph with two weakly connected components.

<img style="position:relative;width:70%" src="http://gellyschool.com/images/tut1_ex3.png" >


In GraphX, we can find all the *weakly* connected components by calling `connectedComponents()` method and the result will be a `Graph` object labeling each component by the *lowest-numbered vertex*. Moreover, since the result is a `Graph`, we can access its `vertices` and `edges` or even compute the number of weakly connected components of our graph like a word count.

In [7]:
val wcc = amazonGraph.connectedComponents()
val numVertWCC = wcc.vertices.map(x => (x._2, 1)).reduceByKey(_ + _)
val numVertLargestWCC = numVertWCC.map(_.swap).sortByKey(ascending=false).first()
println("Number of vertices in the largest WCC: " + numVertLargestWCC._1)
println("Total number of WCC: " + numVertWCC.count())

Number of vertices in the largest WCC: 262111
Total number of WCC: 1


Thus there is only $1$ weakly connected component containing all the vertices and our result verifies the claim about *Nodes in largest WCC* [here](https://snap.stanford.edu/data/amazon0302.html).

It is also possible to find subsets of products that there are direct relations between them (with regard to *directions* of edges). That translates mathematically into [*strongly connected components*](https://en.wikipedia.org/wiki/Strongly_connected_component) (SCC) and in GraphX and we can find them by `stronglyConnectedComponents(numIter=5)`, where `numIter` is the number of iterations required for the computations (as it is a iterative approach, it would be wise to `cache` or `persist` beforehand as we did!). Following the same approach as above, if you want (be careful, it can cosume a lot of resources), you should be able to verify the SCC numbers given [here](https://snap.stanford.edu/data/amazon0302.html).

#### Find the most popular products

Finally, we can use [google's Page Rank algorithm](https://en.wikipedia.org/wiki/PageRank) to measure the **importance of a product** based on the number (and quality) of other products that were bought with it. That is, when multiple products (with high quality) are frequently co-purchased with a product $j,$ then product $j$ recieves high *importance* or *weight*.

<img style="position:relative;width:50%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/2000px-PageRanks-Example.svg.png" >

There are two *types* of page rank implementations in GraphX: *static* and *dynamic*. The difference is the existence of `numIter` parameter in static implementation and tolerance `tol` parameter in dynamic one. We will use the static implementation with $5$ iterations to find top five products with highest weights (importance).

In [8]:
val pr = amazonGraph.staticPageRank(numIter=5)
val productsRank = pr.vertices.map(_.swap).sortByKey(ascending=false).map(_.swap)
println("Five most important product ids and their weights:")
productsRank.take(5).foreach(println)

Five most important product ids and their weights:
(226097,2.26084852668421)
(249921,2.2157186016067265)
(226763,2.163971321491605)
(234407,2.1453611838734403)
(118901,2.125209843612295)


### Next steps

There are number of ways to dig more into the data. For example,

* Given a product id (and its weight), find those products that were co-purchased with it? (*hint:* you can use `collectNeighborIds`, `collectEdges`)


* Which products are co-purchased with at least $k$ other products? (where $k$ is any integer bigger than or equal to $1.$) (*hint:* use `filter` on the `amazonGraph`)


* Explore other [Amazon-network](https://snap.stanford.edu/data/index.html#amazon) datasets separately and answer the same questions about them, or even create a bigger graph of products and attached reviews by joining them all together and explore that.

## Want to learn more?

#### Free courses on [Big Data University](https://bigdatauniversity.com/courses/spark-overview-scala-analytics/?utm_source=tutorial-spark-contest-amazon&utm_medium=dswb&utm_campaign=bdu):

<a href="https://bigdatauniversity.com/courses/spark-overview-scala-analytics/?utm_source=tutorial-spark-contest-amazon&utm_medium=dswb&utm_campaign=bdu"><img src = https://ibm.box.com/shared/static/4w1zym7ek3ujbr9keseck3xi6nqor4iz.png align=left> </a>


#### Complete free [courses on Scala](https://bigdatauniversity.com/learn/scala/?utm_source=tutorial-spark-contest-amazon&utm_medium=dswb&utm_campaign=bdu) and earn [IBM badges](https://bigdatauniversity.com/badges/scala-programming-explorer/?utm_source=tutorial-spark-contest-amazon&utm_medium=dswb&utm_campaign=bdu):    

<a href="https://bigdatauniversity.com/learn/scala/?utm_source=tutorial-spark-contest-amazon&utm_medium=dswb&utm_campaign=bdu"><img src = https://ibm.box.com/shared/static/oxkbzzwgxg9wnr4texux044t42kdnweo.png width=640 align=left> </a>